In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_excel('11.2 WTI.xlsx', 'Spot', index_col='Date')
df2 = pd.read_excel('11.2 WTI.xlsx', 'Front', index_col="Date")
df3 = pd.read_excel('11.2 WTI.xlsx', 'Second', index_col="Date")

In [3]:
df = pd.concat([df1,df2,df3],axis=1)
df = df.fillna(method="ffill") # 앞에 값을 가져와 NaN 값을 덮어씀
df = df.fillna(method="bfill") # 뒤에 값을 가져와 NaN 값을 덮어씀

In [4]:
# 엑셀에서 WTI 만기일 정보 읽어오기
df_e = pd.read_excel('11.2 WTI.xlsx', 'Expiry')
expiry = pd.Series(pd.to_datetime(df_e['Last Trade']))

In [5]:
# 시세 데이터프레임에 최종거래일 표시하기
df["Expiry"] = np.nan
df.head()

,Spot,Front,Second,Expiry
Date,,,,
2017-12-28,59.84,59.84,59.87,NaN
2017-12-27,59.64,59.64,59.69,NaN
2017-12-26,59.97,59.97,60.00,NaN
2017-12-22,58.42,58.47,58.54,NaN
2017-12-21,58.33,58.36,58.40,NaN


In [6]:
# Trading day 중 만기일 찾아서 표시하기
for d in df.index : # date 값 하나씩 읽어와서
    if d in pd.to_datetime(expiry.values) : # expiry 에 같은 값있으면
        df.loc[d].Expiry =1 # expiry 열에 1이라고 표시
df[5:10]

,Spot,Front,Second,Expiry
Date,,,,
2017-12-20,58.02,58.09,58.13,NaN
2017-12-19,57.46,57.46,57.56,1.0
2017-12-18,57.16,57.16,57.22,NaN
2017-12-15,57.30,57.30,57.33,NaN
2017-12-14,57.04,57.04,57.08,NaN


In [8]:
#현물에 투자했다면
# 손익 = 현재가격 - 초기가격 
price_now = float(df['2017-12-28']['Spot'])
price_before = float(df['2010-01-04']['Spot'])
profit = (price_now - price_before) *1000
print('손익 (현재가격 - 초기가격) : ',"{:,.2f}".format(profit))

손익 (현재가격 - 초기가격) :  -21,670.00


In [10]:
# 선물에 투자했다면
# 현재가격 - 초기가격 -롤오버 비용
roll_over_cost_acc = 0 # 롤오버 비용 누적값 초기화
for d in df.index: # df 일자별로 순환하며
    if (df.loc[d].Expiry == 1): # 최종거래일이면 롤오버 계산 시작
        #롤오버 비용 = 차근월물가격 - 최근 월물 가격
        roll_over_cost = df.loc[d].Second - df.loc[d].Front # 롤오버이용
        roll_over_cost_acc = roll_over_cost_acc + roll_over_cost # 누적값
print('누적 롤오버 비용 :','{:,.2f}'.format(roll_over_cost_acc))

누적 롤오버 비용 : 40.68


In [11]:
price_now = float(df['2017-12-28']['Front']) # 선물 현재가
price_before = float(df['2010-01-04']['Front']) # 선물 최초가
profit = (price_now - price_before - roll_over_cost_acc) *1000 
print('누적손익 :','{:,.2f}'.format(profit))

누적손익 : -62,350.00
